In [1]:
import json
import requests
from requests.auth import HTTPBasicAuth
from twilio.rest import Client
import time
import sys
import logging
import os
import glob
import datetime

logger = logging.getLogger(__name__)

In [35]:
account_sid = 'AC44ad7addd81382eb21f19cecfe8c5634'
#account_sid = 'ACea876589f9b5d027d0c069f21e50b8be'
auth_token = '1ef1718fcc720eeea5bacdb7a9a73677'
#auth_token = '9ba91fafc9ee302d5f02d1ec1541fed8'
plans_to_check = ['Test-A','Val-A']
from_number = '+13236901080'
to_numbers = ['+13236903633','+14016543129']

In [10]:
res = requests.get('http://34.229.165.221:8085/rest/api/latest/result.json?os_authType=basic', 
                   auth=HTTPBasicAuth('testuser', 'testpassword'))
result = json.loads(res.content.decode())

In [11]:
def send_message(client, body_message, from_number, to_numbers):
    for number in to_numbers:
        message = client.messages \
        .create(
             body=body_message,
             from_=from_number,
             to=number
         )

        logger.info('Message sent to : ',number,' SID : ',message.sid)

def call(client, from_number, to_numbers):
    for number in to_numbers:
        call = client.calls.create(
                    url='http://demo.twilio.com/docs/voice.xml',
                    to=number,
                    from_=from_number
                )

        logger.info('Called number : ',number,' SID : ',call.sid)
            
for plan in result['results']['result']:
    client = Client(account_sid, auth_token)
    if (plan['state'] == 'Failed' or plan['buildState'] == 'Failed') and plan['plan']['shortName'] in plans_to_check:
        print('Plan %s has failed. Sending Message'%plan['plan']['shortName'])
        body_message = '%s has failed.'%plan['plan']['shortName']
        send_message(client, body_message, from_number, to_numbers)
    elif (plan['state'] == 'Success' or plan['buildState'] == 'Success') and plan['plan']['shortName'] in plans_to_check:
        log_file_location = ''
        error_check_flag = False
        err_count = 0
        error_line_locations = []
        with open(log_file_location, 'r') as f:
            for i, line in enumerate(f):
                if 'an error has occured' in line or 'process terminated' in line:
                    error_check_flag = True
                    err_count += 1
                    error_line_locations.append(i+1)
                    
        if error_check_flag:        
            body_message = '%s has failures evethough build succeeded. Error Count : %d. Error Line Locations : %s'\
                %(plan['plan']['shortName'],err_count, str(error_line_locations))
            send_message(client, body_message, from_number, to_numbers)

Plan Test-A has failed. Sending Message
Message sent to :  +13236903633  SID :  SM620204210f7648708f51938f9f228a70
Message sent to :  +14016543129  SID :  SM7c0ccd1ab5c64c09b421ae922fb00955
Plan Val-A has failed. Sending Message
Message sent to :  +13236903633  SID :  SM8bc3924445d9467fb72ee95bceef33b7
Message sent to :  +14016543129  SID :  SMb3a464fe6e7747278bbfda209cd06922


In [23]:
log_location = os.getcwd()
log_location

'/home/sunny/Desktop'

In [34]:
def send_message(client, body_message, from_number, to_numbers):
    for number in to_numbers:
        message = client.messages \
        .create(
             body=body_message,
             from_=from_number,
             to=number
         )

        logger.info('Message sent to : ',number,' SID : ',message.sid)

client = Client(account_sid, auth_token)        
latest_file = os.listdir(log_location)[0]
latest_file_stat = os.stat(latest_file)
try:
    latest_time = datetime.datetime.fromtimestamp(latest_file_stat.st_mtime)
except:
    latest_time = datetime.datetime.fromtimestamp(latest_file_stat.st_stime)
for file in os.listdir(log_location)[1:]:
    #print(os.stat(file))
    stats = os.stat(file)
    try:
        other_time = datetime.datetime.fromtimestamp(stats.st_mtime)
    except:
        other_time = datetime.datetime.fromtimestamp(stats.st_stime)
    if other_time  > latest_time:
        latest_time = other_time
        latest_file = file
        
error_check_flag = False
err_count = 0
error_line_locations = []
with open(os.path.join(log_location, latest_file), 'r') as f:
    for i, line in enumerate(f):
        if 'an error has occured' in line or 'process terminated' in line:
            error_check_flag = True
            err_count += 1
            error_line_locations.append(i+1)
    if error_check_flag:        
            body_message = '%s has failures. Error Count : %d. Error Line Locations : %s'\
                %(os.path.join(log_location, latest_file),err_count, str(error_line_locations))
            send_message(client, body_message, from_number, to_numbers)

In [37]:
datetime.datetime.fromtimestamp(os.path.getmtime(latest_file))

datetime.datetime(2019, 4, 2, 10, 28, 42, 819107)